In [ ]:
# !pip install opencv-python

In [ ]:
import cv2
import numpy as np
import time

# Paths to the required files
weights_path = 'Assets/Trained_Model/yolov3/yolov3.weights'
config_path = 'Assets/Trained_Model/yolov3/yolov3.cfg'
names_path = 'Assets/Trained_Model/coco.names'
traffic_light_weights_path = 'Assets/Trained_Model/yolov3/yolov3_10000.weights'
traffic_light_config_path = 'Assets/Trained_Model/yolov3/yolov3.cfg'
traffic_light_names_path = 'Assets/Structured files/obj.names'

# Read the class labels from coco.names and traffic_light.names
with open(names_path, 'r') as f:
    classes = f.read().strip().split('\n')

with open(traffic_light_names_path, 'r') as f:
    traffic_light_classes = f.read().strip().split('\n')

# Load the YOLO models
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
traffic_light_net = cv2.dnn.readNetFromDarknet(traffic_light_config_path, traffic_light_weights_path)

def detect_objects(frame, net, classes, confidence_threshold=0.5, nms_threshold=0.4):
    (H, W) = frame.shape[:2]
    
    # Create a blob and pass it through the network
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    
    outputs = net.forward(output_layers)
    
    boxes = []
    confidences = []
    class_ids = []
    
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_threshold:
                label = classes[class_id]
                if label in ['car', 'motorbike', 'bus', 'RedLight', 'GreenLight']:
                    box = detection[0:4] * np.array([W, H, W, H])
                    (centerX, centerY, width, height) = box.astype("int")
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
    
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)
    
    results = []
    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            if classes[class_ids[i]] == 'GreenLight':
                color = [0, 128, 0]  # Green color for GreenLight
            elif classes[class_ids[i]] == 'RedLight':
                color = [0, 0, 255]  # Red color for RedLight
            else:
                color = [0, 255, 0]  # Default color for other labels
            
            results.append((x, y, w, h, classes[class_ids[i]], confidences[i], color))
    
    return results

# Path to the video
video_path = 'Assets\Traffic_Data\Traffic_Violations\Violation_Video\Violation 2 - Made with Clipchamp.mp4'

# Open the video
cap = cv2.VideoCapture(video_path)

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

# Define the line coordinates for the red light line
red_light_line_y1 = 480  # Top Y-coordinate of the red light line
red_light_line_y2 = 440  # Bottom Y-coordinate of the red light line
intersection_line_start_x = 0  # Starting X-coordinate of the red light line
intersection_line_end_x = 800  # Ending X-coordinate of the red light line

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    start_time = time.time()  # Start time for frame processing
    
    # Object detection with the main YOLO model
    objects = detect_objects(frame, net, classes)
    
    # Traffic light detection with the custom trained YOLO model
    traffic_lights = detect_objects(frame, traffic_light_net, traffic_light_classes)
    
    # Determine the state of the traffic light
    red_light = any(label == 'red_light' for _, _, _, _, label, _, _ in traffic_lights)
    
    # Draw the detected objects
    for (x, y, w, h, label, confidence, color) in objects + traffic_lights:
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # Check for violation when crossing the intersection line
        if label in ['car', 'motorbike', 'bus'] and y < red_light_line_y2 and y >= 300:
            cv2.putText(frame, "Violation!", (x, y - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    # Draw the shortened red light line
    cv2.line(frame, (intersection_line_start_x, red_light_line_y1), (intersection_line_end_x, red_light_line_y2), (0, 0, 255), 2)
    
    # Show the resulting frame
    cv2.imshow('Video', frame)
    
    elapsed_time = time.time() - start_time
    print(f"Processing Time: {elapsed_time:.2f} seconds")
    
    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources and close windows
cap.release()
cv2.destroyAllWindows()
